<a href="https://colab.research.google.com/github/Kuzay3t/3D_Image_Reconstruction/blob/main/3d_Reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import zipfile
import os
from google.colab import drive
import zipfile
import open3d as o3d
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(os.listdir('/content/drive/My Drive'))

['FINANCIAL LITERACY CERTIFICATE.pdf', 'KUZAYET BMC.pdf', 'GST 208 RECEIPT.pdf', 'VID-20240508-WA0014.mp4', 'BAGAI GLORY RESUME.pdf', 'BAGAI GLORY HEADSHOT.jpeg', '300LVL 2ND SEMESTER', 'project proposal template', 'RENUE DOCUMENT', 'Colab Notebooks', 'archive (1).zip', 'archive.zip']


In [ ]:
print(os.listdir('/content/ModelNet10'))

['ModelNet10', 'metadata_modelnet10.csv']


In [ ]:
zip_path = '/content/drive/My Drive/archive.zip'


In [ ]:
extract_path = '/content/ModelNet10'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
# Set path to your unzipped dataset
root_dir = '/content/ModelNet10/ModelNet10'
category = 'chair'  # Example class; change as needed
model_dir = os.path.join(root_dir, category, 'train')
sample_file = [f for f in os.listdir(model_dir) if f.endswith('.off')][0]
file_path = os.path.join(model_dir, sample_file)

In [ ]:
# Function to load .off file as point cloud
def load_off_as_pointcloud(filename, n_points=1024):
    mesh = o3d.io.read_triangle_mesh(filename)
    pcd = mesh.sample_points_uniformly(number_of_points=n_points)
    return np.asarray(pcd.points)


In [ ]:
# Example: Load one point cloud
clean_points = load_off_as_pointcloud(file_path)

In [ ]:
# adding noise to the dataset

def add_noise(points, std=0.02):
    noise = np.random.normal(scale=std, size=points.shape)
    return points + noise

noisy_points = add_noise(clean_points)

In [27]:
# denoising the image

class DenoiseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )
    def forward(self, x):
        return self.mlp(x)

model = DenoiseNet()

In [28]:
# preparing the data for training
# Convert to torch tensors
noisy_tensor = torch.tensor(noisy_points, dtype=torch.float32)
clean_tensor = torch.tensor(clean_points, dtype=torch.float32)

# For a toy example, batch size is the number of points
dataset = [(noisy_tensor, clean_tensor)]

In [31]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

In [32]:
# training the model

model.train()
for epoch in range(200):  # Toy loop, real training would use more data
    optimizer.zero_grad()
    pred = model(noisy_tensor)
    loss = loss_fn(pred, clean_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 20 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.6f}")

Epoch 20, Loss: 51.974472
Epoch 40, Loss: 11.516495
Epoch 60, Loss: 2.132057
Epoch 80, Loss: 0.613787
Epoch 100, Loss: 0.312459
Epoch 120, Loss: 0.187423
Epoch 140, Loss: 0.118696
Epoch 160, Loss: 0.083146
Epoch 180, Loss: 0.064508
Epoch 200, Loss: 0.053015


In [33]:
# Predict denoised points
model.eval()
with torch.no_grad():
    denoised = model(noisy_tensor).numpy()

In [34]:
# Visualize using open3d
pcd_noisy = o3d.geometry.PointCloud()
pcd_noisy.points = o3d.utility.Vector3dVector(noisy_points)
pcd_denoised = o3d.geometry.PointCloud()
pcd_denoised.points = o3d.utility.Vector3dVector(denoised)

In [35]:
o3d.visualization.draw_geometries([pcd_noisy], window_name='Noisy')
o3d.visualization.draw_geometries([pcd_denoised], window_name='Denoised')

[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
